In [190]:
import stata_setup
stata_setup.config("/Applications/Stata 17", "be")

### Import the Distance to US Capital Data

*Note: Country ISO Codes and missing data added to FRIET Country distance file*

In [191]:
%%stata
frame create distUs
cwf distUs


. frame create distUs

. cwf distUs

. 


In [192]:
%%stata
import excel "Data/distToUS.xlsx", sheet("Sheet1") firstrow
describe


. import excel "Data/distToUS.xlsx", sheet("Sheet1") firstrow
(4 vars, 219 obs)

. describe

Contains data
 Observations:           219                  
    Variables:             4                  
-------------------------------------------------------------------------------
Variable      Storage   Display    Value
    name         type    format    label      Variable label
-------------------------------------------------------------------------------
countryName     str32   %32s                  countryName
iso3Code        str3    %9s                   iso3Code
overSeasTerr    str3    %9s                   overSeasTerr
distanceKm      double  %10.0g                distanceKm
-------------------------------------------------------------------------------
Sorted by: 
     Note: Dataset has changed since last saved.

. 


In [193]:
%%stata
label variable distanceKm "distance from USA Capital to Country Capital"
label variable iso3Code "ISO-3 Character Country Code"
label variable overSeasTerr "Country Code of Country this observation is an teritory of"
describe


. label variable distanceKm "distance from USA Capital to Country Capital"

. label variable iso3Code "ISO-3 Character Country Code"

. label variable overSeasTerr "Country Code of Country this observation is an t
> eritory of"

. describe

Contains data
 Observations:           219                  
    Variables:             4                  
-------------------------------------------------------------------------------
Variable      Storage   Display    Value
    name         type    format    label      Variable label
-------------------------------------------------------------------------------
countryName     str32   %32s                  countryName
iso3Code        str3    %9s                   ISO-3 Character Country Code
overSeasTerr    str3    %9s                   Country Code of Country this
                                                observation is an teritory of
distanceKm      double  %10.0g                distance from USA Capital to
                           

In [194]:
%%stata
cwf distUs
duplicates tag iso3Code, generate(dup)
list countryName iso3Code if dup==1


. cwf distUs

. duplicates tag iso3Code, generate(dup)

Duplicates in terms of iso3Code

. list countryName iso3Code if dup==1

. 


*Duplicates adjusted in the source file*

---

## Import Languages and Link to Distance Data

In [195]:
%%stata
frame create lang
cwf lang
import excel "Data/languages.xlsx", sheet("Sheet1") firstrow
describe


. frame create lang

. cwf lang

. import excel "Data/languages.xlsx", sheet("Sheet1") firstrow
(4 vars, 164 obs)

. describe

Contains data
 Observations:           164                  
    Variables:             4                  
-------------------------------------------------------------------------------
Variable      Storage   Display    Value
    name         type    format    label      Variable label
-------------------------------------------------------------------------------
langCode        byte    %10.0g                langCode
langName        str14   %14s                  langName
countryName     str32   %32s                  countryName
iso3Code        str3    %9s                   iso3Code
-------------------------------------------------------------------------------
Sorted by: 
     Note: Dataset has changed since last saved.

. 


In the languages file, English == 5.

In [196]:
%%stata
gen common_lang = 1 if langCode==5
list if common_lang == 1


. gen common_lang = 1 if langCode==5
(126 missing values generated)

. list if common_lang == 1

     +----------------------------------------------------------------------+
     | langCode   langName                countryName   iso3Code   common~g |
     |----------------------------------------------------------------------|
  6. |        5    English                  Australia        AUS          1 |
  8. |        5    English                    Bahamas        BHS          1 |
 11. |        5    English                   Barbados        BRB          1 |
 13. |        5    English                     Belize        BLZ          1 |
 15. |        5    English                    Bermuda        BMU          1 |
     |----------------------------------------------------------------------|
 26. |        5    English                     Canada        CAN          1 |
 27. |        5    English             Cayman Islands        CYM          1 |
 49. |        5    English                  

In [197]:
%%stata
replace common_lang=0 if common_lang==.

In [198]:
%%stata
tab common_lang


common_lang |      Freq.     Percent        Cum.
------------+-----------------------------------
          0 |        126       76.83       76.83
          1 |         38       23.17      100.00
------------+-----------------------------------
      Total |        164      100.00


In [199]:
%%stata
duplicates tag iso3Code, generate(dup)
list countryName iso3Code if dup==1


. duplicates tag iso3Code, generate(dup)

Duplicates in terms of iso3Code

. list countryName iso3Code if dup==1

. 


In [200]:
%%stata
drop if iso3Code==""

(12 observations deleted)


iso3Code is unique in the language dataset

---

## Import 2014 US Exports data

In [201]:
%%stata
frame create usExports
cwf usExports
import excel "Data/us_merch_exports_2014.xlsx", sheet("USEXPORTS") firstrow
describe


. frame create usExports

. cwf usExports

. import excel "Data/us_merch_exports_2014.xlsx", sheet("USEXPORTS") firstrow
(3 vars, 219 obs)

. describe

Contains data
 Observations:           219                  
    Variables:             3                  
-------------------------------------------------------------------------------
Variable      Storage   Display    Value
    name         type    format    label      Variable label
-------------------------------------------------------------------------------
country         str32   %32s                  country
iso3Code        str3    %9s                   iso3Code
usExp           double  %10.0g                usExp
-------------------------------------------------------------------------------
Sorted by: 
     Note: Dataset has changed since last saved.

. 


**Check for duplicates or other data errors.**

In [202]:
%%stata
cwf usExports
duplicates tag iso3Code, generate(dup)
list country iso3Code if dup==1


. cwf usExports

. duplicates tag iso3Code, generate(dup)

Duplicates in terms of iso3Code

. list country iso3Code if dup==1

. 


**All iso3Codes values are now unique**

---

## Import 2014 GDP Data

In [203]:
%%stata
frame create gdp2014
cwf gdp2014


. frame create gdp2014

. cwf gdp2014

. 


In [204]:
%%stata
import excel "Data/gdpByCountry2014.xlsx", sheet("Sheet1") firstrow
describe
duplicates list iso3Code


. import excel "Data/gdpByCountry2014.xlsx", sheet("Sheet1") firstrow
(3 vars, 264 obs)

. describe

Contains data
 Observations:           264                  
    Variables:             3                  
-------------------------------------------------------------------------------
Variable      Storage   Display    Value
    name         type    format    label      Variable label
-------------------------------------------------------------------------------
country         str52   %52s                  country
iso3Code        str3    %9s                   iso3Code
gdp             double  %10.0g                gdp
-------------------------------------------------------------------------------
Sorted by: 
     Note: Dataset has changed since last saved.

. duplicates list iso3Code

Duplicates in terms of iso3Code

(0 observations are duplicates)

. 


---

## Create links between dataframes and generate a master dataset

In [205]:
%%stata
frame dir


  default    0 x 0
* distUs     219 x 5
* gdp2014    264 x 3
* lang       152 x 6
* usExports  219 x 4

Note: Frames marked with * contain unsaved data.


#### Link GDP 2014 File

In [206]:
%%stata
cwf distUs
frlink 1:1 iso3Code, frame(gdp2014)
frget gdp, from(gdp2014)
list country iso3Code gdp if gdp==.


. cwf distUs

. frlink 1:1 iso3Code, frame(gdp2014)
  (17 observations in frame distUs unmatched)

. frget gdp, from(gdp2014)
(29 missing values generated)
  (1 variable copied from linked frame)

. list country iso3Code gdp if gdp==.

     +-------------------------------------------------+
     |                    countryName   iso3Code   gdp |
     |-------------------------------------------------|
  6. |                       Anguilla        AIA     . |
 10. |                          Aruba        ABW     . |
 22. |                        Bermuda        BMU     . |
 28. | British Indian Ocean Territory        IOT     . |
 29. |         British Virgin Islands        VGB     . |
     |-------------------------------------------------|
 40. |                 Cayman Islands        CYM     . |
 45. |               Christmas Island        CXR     . |
 46. |        Cocos (Keeling) Islands        CCK     . |
 50. |                   Cook Islands        COK     . |
 54. |                

#### Link Language File

In [207]:
%%stata
cwf distUs
frlink 1:1 iso3Code, frame(lang)
frget common_lang, from(lang)
list country iso3Code common_lang if common_lang==.


. cwf distUs

. frlink 1:1 iso3Code, frame(lang)
  (68 observations in frame distUs unmatched)

. frget common_lang, from(lang)
(68 missing values generated)
  (1 variable copied from linked frame)

. list country iso3Code common_lang if common_lang==.

     +--------------------------------------------------------+
     |                      countryName   iso3Code   common~g |
     |--------------------------------------------------------|
  4. |                          Andorra        AND          . |
  6. |                         Anguilla        AIA          . |
  7. |              Antigua and Barbuda        ATG          . |
  9. |                          Armenia        ARM          . |
 10. |                            Aruba        ABW          . |
     |--------------------------------------------------------|
 13. |                       Azerbaijan        AZE          . |
 18. |                          Belarus        BLR          . |
 25. |           Bosnia and Herzegovina  

In [208]:
%%stata
* Set Missing Values to 0, not English
replace common_lang=0 if common_lang==.


. * Set Missing Values to 0, not English
. replace common_lang=0 if common_lang==.
(68 real changes made)

. 


#### Link US Exports File

In [209]:
%%stata
frlink 1:1 iso3Code, frame(usExports)
frget usExp=usExp, from (usExports)
list country iso3Code gdp if usExports==.


. frlink 1:1 iso3Code, frame(usExports)
  (all observations in frame distUs matched)

. frget usExp=usExp, from (usExports)
  (1 variable copied from linked frame)

. list country iso3Code gdp if usExports==.

. 


In [210]:
%%stata
describe


Contains data
 Observations:           219                  
    Variables:            11                  
-------------------------------------------------------------------------------
Variable      Storage   Display    Value
    name         type    format    label      Variable label
-------------------------------------------------------------------------------
countryName     str32   %32s                  countryName
iso3Code        str3    %9s                   ISO-3 Character Country Code
overSeasTerr    str3    %9s                   Country Code of Country this
                                                observation is an teritory of
distanceKm      double  %10.0g                distance from USA Capital to
                                                Country Capital
dup             byte    %12.0g                
gdp2014         int     %10.0g                
gdp             double  %10.0g                gdp
lang            int     %10.0g                
common_lang  

In [211]:
%%stata
label variable common_lang "=1 if English is the langauge of the country. Otherwise =0"
label variable usExp "Value of US Exports to country in 2014"
label variable gdp "Country GDP in 2014"
describe


. label variable common_lang "=1 if English is the langauge of the country. Oth
> erwise =0"

. label variable usExp "Value of US Exports to country in 2014"

. label variable gdp "Country GDP in 2014"

. describe

Contains data
 Observations:           219                  
    Variables:            11                  
-------------------------------------------------------------------------------
Variable      Storage   Display    Value
    name         type    format    label      Variable label
-------------------------------------------------------------------------------
countryName     str32   %32s                  countryName
iso3Code        str3    %9s                   ISO-3 Character Country Code
overSeasTerr    str3    %9s                   Country Code of Country this
                                                observation is an teritory of
distanceKm      double  %10.0g                distance from USA Capital to
                                                Count

In [212]:
%%stata
* Drop the link variables from the dataset
drop gdp2014 usExports lang


. * Drop the link variables from the dataset
. drop gdp2014 usExports lang

. 


In [213]:
%%stata
describe


Contains data
 Observations:           219                  
    Variables:             8                  
-------------------------------------------------------------------------------
Variable      Storage   Display    Value
    name         type    format    label      Variable label
-------------------------------------------------------------------------------
countryName     str32   %32s                  countryName
iso3Code        str3    %9s                   ISO-3 Character Country Code
overSeasTerr    str3    %9s                   Country Code of Country this
                                                observation is an teritory of
distanceKm      double  %10.0g                distance from USA Capital to
                                                Country Capital
dup             byte    %12.0g                
gdp             double  %10.0g                Country GDP in 2014
common_lang     float   %9.0g                 =1 if English is the langauge of
            

### Create natural log forms if distanceKm, usExp, and gdp

In [214]:
%%stata
generate ldist = log(distanceKm)
label variable ldist "ln(distanceKm)"
generate lUsExp = log(usExp)
label variable lUsExp "ln(usExp)"
generate lGdp = log(gdp)
label variable lGdp "ln(gdp)"
describe


. generate ldist = log(distanceKm)
(10 missing values generated)

. label variable ldist "ln(distanceKm)"

. generate lUsExp = log(usExp)

. label variable lUsExp "ln(usExp)"

. generate lGdp = log(gdp)
(29 missing values generated)

. label variable lGdp "ln(gdp)"

. describe

Contains data
 Observations:           219                  
    Variables:            11                  
-------------------------------------------------------------------------------
Variable      Storage   Display    Value
    name         type    format    label      Variable label
-------------------------------------------------------------------------------
countryName     str32   %32s                  countryName
iso3Code        str3    %9s                   ISO-3 Character Country Code
overSeasTerr    str3    %9s                   Country Code of Country this
                                                observation is an teritory of
distanceKm      double  %10.0g                distance from USA 

### Add Trade Agreement Indicator variables for Jordan and Chile

Examine impact of the trade agreements
* US-Jordan [JOR]. Agreement in place since 2001.
* US-Chile [CHL]. Agreement in place since 2004.

In [215]:
%%stata
generate tradeAgreement = iso3Code=="CHL" |iso3Code== "JOR"
label variable tradeAgreement "=1 if USA has bilateral trade agreement, JOR & CHL only. =0 otherwise"
tab tradeAgreement
list countryName iso3Code usExp gdp distanceKm tradeAgreement ///
      if tradeAgreement==1


. generate tradeAgreement = iso3Code=="CHL" |iso3Code== "JOR"

. label variable tradeAgreement "=1 if USA has bilateral trade agreement, JOR &
>  CHL only. =0 otherwise"

. tab tradeAgreement

  =1 if USA |
        has |
  bilateral |
      trade |
 agreement, |
  JOR & CHL |
   only. =0 |
  otherwise |      Freq.     Percent        Cum.
------------+-----------------------------------
          0 |        217       99.09       99.09
          1 |          2        0.91      100.00
------------+-----------------------------------
      Total |        219      100.00

. list countryName iso3Code usExp gdp distanceKm tradeAgreement ///
>       if tradeAgreement==1

     +-------------------------------------------------------------------+
     | countr~e   iso3Code       usExp         gdp   distan~m   tradeA~t |
     |-------------------------------------------------------------------|
 43. |    Chile        CHL   1.663e+10   2.606e+11   8079.282          1 |
104. |   Jordan        JOR 

### Add indicator variable for shared border with Mexico and Canada

In [216]:
%%stata
generate sharedBorder = iso3Code =="CAN" | iso3Code =="MEX"
label variable sharedBorder "Trading partner shares a land or small Water border with USA"
tab sharedBorder
list countryName iso3Code usExp gdp distanceKm sharedBorder ///
      if sharedBorder==1


. generate sharedBorder = iso3Code =="CAN" | iso3Code =="MEX"

. label variable sharedBorder "Trading partner shares a land or small Water bor
> der with USA"

. tab sharedBorder

    Trading |
    partner |
   shares a |
    land or |
small Water |
border with |
        USA |      Freq.     Percent        Cum.
------------+-----------------------------------
          0 |        217       99.09       99.09
          1 |          2        0.91      100.00
------------+-----------------------------------
      Total |        219      100.00

. list countryName iso3Code usExp gdp distanceKm sharedBorder ///
>       if sharedBorder==1

     +-------------------------------------------------------------------+
     | countr~e   iso3Code       usExp         gdp   distan~m   shared~r |
     |-------------------------------------------------------------------|
 38. |   Canada        CAN   3.121e+11   1.799e+12    733.894          1 |
131. |   Mexico        MEX   2.403e+11   1.314e+12   3038.

In [217]:
%%stata
save "Data/gravityModelData", replace


file Data/gravityModelData.dta saved


In [218]:
%%stata
frame dir

  default    0 x 0
  distUs     219 x 13; gravityModelData.dta
* gdp2014    264 x 3
* lang       152 x 6
* usExports  219 x 4

Note: Frames marked with * contain unsaved data.


In [219]:
%%stata
cwf default
frame drop distUs
frame drop gdp2014
frame drop lang
frame drop usExports
frame dir


. cwf default

. frame drop distUs

. frame drop gdp2014

. frame drop lang

. frame drop usExports

. frame dir
  default  0 x 0

. 


In [220]:
%%stata
clear

# END DATA Prep